In [ ]:
import numpy as np 
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


In [ ]:
#----------------------------------------->>>>
df=pd.read_csv("PGA.csv")
df.head(10)

In [ ]:
print("THE Relation of Magnitude with PGA",df.Magnitude.corr(df.PGA_target))
print("THE Relation of distance with PGA",df.distance.corr(df.PGA_target))

In [ ]:
# Scale the magnitude and distance features
scaler = MinMaxScaler()
df[['Magnitude', 'distance']] = scaler.fit_transform(df[['Magnitude', 'distance']])
df

In [ ]:
df = pd.get_dummies(df, columns=['site_class'],drop_first=False)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = df[['Magnitude', 'distance','site_class_A','site_class_B','site_class_C']]
y = df['PGA_target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from tensorflow.keras import layers, regularizers
kernel=regularizers.l2(0.01)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(5,)),
    tf.keras.layers.Dense(2, activation='tanh',kernel_regularizer=kernel),
    tf.keras.layers.Dense(1, activation='linear')
])


In [ ]:
model.summary()

In [ ]:
optimizer_01=tf.keras.optimizers.SGD(learning_rate=0.001,name='SGD')
optimizer_02=tf.keras.optimizers.legacy.Adam(lr=0.001,name='Adam')

model.compile(
    
            optimizer=optimizer_02,
            loss= tf.keras.losses.MeanSquaredError(),
            metrics=['accuracy'],
    
              
             
             )



In [ ]:
from keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)


#y_train = y_train.astype('float32')

model.fit(X_train_scaled,y_train,
          batch_size=6,
          epochs=30,
          #validation_split=0.2,
          shuffle=True,
          #callbacks=[early_stopping],
          verbose=1,
)


In [ ]:
new_input_data1 = [[7.6, 36]]
predicted_log_pga1 = model.predict(X_test_scaled)
print ("The prediction of test \n ",X_test_scaled," \n \n  And the predicted value is \n ",predicted_log_pga1)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Assuming you have the y_test as a pandas Series object
y_test_array = y_test.values.reshape(-1, 1)  # Convert y_test to a NumPy array and reshape it

y_test_normalized = scaler.fit_transform(y_test_array)


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the predicted values 'predicted_log_pga1' and the corresponding test labels 'y_test'
predicted_log_pga1 = predicted_log_pga1.flatten()  # Flatten the predicted values to match the shape of y_test

plt.plot(y_test_normalized, "o", label=" y _test  Labels")
plt.plot(predicted_log_pga1,"o", label="Predicted Values on y_test set ")
plt.xlabel(" Original Target_PGA in test set ")
plt.ylabel("Observed or predicted Target_PGA for the test set")
plt.legend()
plt.show()
plt.savefig("PGA_Prediction.jpg")


In [ ]:

predicted_log_pga2 = model.predict(X_train_scaled)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Assuming you have the y_test as a pandas Series object
y_train_array = y_train.values.reshape(-1, 1)  # Convert y_test to a NumPy array and reshape it

y_train_normalized = scaler.fit_transform(y_train_array)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the predicted values 'predicted_log_pga1' and the corresponding test labels 'y_test'
predicted_log_pga2 = predicted_log_pga2.flatten()  # Flatten the predicted values to match the shape of y_test

plt.plot(y_train_normalized, "o", label=" y _test  Labels")
plt.plot(predicted_log_pga2,"o", label="Predicted Values on y_train set ")
plt.xlabel(" Original Target_PGA in train set ")
plt.ylabel("Observed or predicted Target_PGA for the train set")
plt.legend()
plt.show()


In [ ]:
# Assuming you have trained your model and have the X_test and y_test data prepared

# Evaluate the model on the test set
loss, accuracy, mse = model.evaluate(X_test_scaled, y_test)

# Print the accuracy
print("Test Accuracy:", accuracy)
